In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

def create_animation(ims):
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(ims[len(ims)//2], cmap="gray")

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

In [ ]:
import torch
from matplotlib import pyplot as plt

from train import Trainer
from config import Config
from util import visualize

conf = Config()
device = torch.device('cpu')
conf.batch_size = 4
input_dir = '../input'
trainer = Trainer(conf, input_dir, device, 1, None)
loader = trainer.train_loader
batch = iter(loader).next()
images, labels = batch['img'], batch['msk']

# the data is in NCWHD format
middle = images.shape[-1]//2
images = images.numpy()[..., middle - 5:middle + 5]
labels = labels.numpy()[...,  middle - 5:middle + 5]
anims = []
for image, label in zip(images, labels):
    # the data is CWHD format
    image = image.transpose((3, 2, 1, 0))
    label = label.transpose((3, 2, 1, 0))
    image -= image.min()
    image /= image.max()  
    
    anims.append(create_animation(image))
    anims.append(create_animation(label))
    

In [ ]:
from ipywidgets import Output, GridspecLayout
from IPython import display

grid = GridspecLayout(len(anims), 1)
for i, anim in enumerate(anims):
    train_out = Output()
    with train_out:
        display.display(anim)
    grid[i, 0] = train_out

grid